<a href="https://colab.research.google.com/github/SamAbr/FPL-Squad-Selection/blob/main/FPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import requests
import pandas as pd

# Get FPL data
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
data = requests.get(url).json()

# Convert to DataFrame
players = pd.DataFrame(data['elements'])
teams = pd.DataFrame(data['teams'])
for column in players.columns:
  print(column)

can_transact
can_select
chance_of_playing_next_round
chance_of_playing_this_round
code
cost_change_event
cost_change_event_fall
cost_change_start
cost_change_start_fall
dreamteam_count
element_type
ep_next
ep_this
event_points
first_name
form
id
in_dreamteam
news
news_added
now_cost
photo
points_per_game
removed
second_name
selected_by_percent
special
squad_number
status
team
team_code
total_points
transfers_in
transfers_in_event
transfers_out
transfers_out_event
value_form
value_season
web_name
region
team_join_date
birth_date
has_temporary_code
opta_code
minutes
goals_scored
assists
clean_sheets
goals_conceded
own_goals
penalties_saved
penalties_missed
yellow_cards
red_cards
saves
bonus
bps
influence
creativity
threat
ict_index
clearances_blocks_interceptions
recoveries
tackles
defensive_contribution
starts
expected_goals
expected_assists
expected_goal_involvements
expected_goals_conceded
influence_rank
influence_rank_type
creativity_rank
creativity_rank_type
threat_rank
threat_rank_

In [3]:
players['price'] = players['now_cost'] / 10
players['points_per_90'] = players['total_points'] / (players['minutes'] / 90)
players['value'] = players['total_points'] / players['price']

In [5]:
best_value = players.sort_values('value', ascending=False).head(10)
top_midfielders = players[(players['element_type']==3) & (players['minutes']>900)] \
                     .sort_values('points_per_90', ascending=False).head(10)

best_value

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,defensive_contribution_per_90,price,points_per_90,value
556,True,True,NaN,NaN,223827,0,0,0,0,1,...,1,1,96,33,1.00,1.00,13.00,4.5,17.000000,3.777778
6,True,True,NaN,NaN,466075,0,0,0,0,1,...,3,2,136,49,1.27,1.27,6.34,5.5,16.478873,2.363636
239,True,True,NaN,NaN,577016,0,0,0,0,0,...,13,4,181,69,1.00,1.00,8.00,4.0,9.000000,2.250000
429,True,True,NaN,NaN,477064,0,0,0,0,1,...,7,3,126,45,1.38,1.38,8.31,5.0,15.230769,2.200000
125,True,True,NaN,NaN,437730,1,-1,1,-1,1,...,2,1,39,11,1.00,0.00,5.00,7.1,15.000000,2.112676
627,True,True,NaN,NaN,212319,0,0,0,0,1,...,4,1,75,14,1.27,1.27,6.34,6.5,16.478873,2.000000
0,True,True,NaN,NaN,154561,0,0,0,0,1,...,9,1,26,4,1.00,1.00,0.00,5.5,10.000000,1.818182
208,True,True,NaN,NaN,219249,0,0,0,0,1,...,10,2,191,63,1.03,1.03,3.10,5.5,10.344828,1.818182
595,True,True,NaN,NaN,184254,0,0,0,0,0,...,16,3,76,10,1.00,1.00,0.00,5.0,9.000000,1.800000
263,True,True,NaN,NaN,244723,0,0,0,0,0,...,17,6,156,56,1.00,1.00,13.00,5.0,9.000000,1.800000


In [7]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.3 MB/s eta 0:00:00


In [9]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define problem
model = LpProblem("FPL_Team_Selection", LpMaximize)

# Decision variables: 1 = pick player, 0 = not picked
player_vars = {i: LpVariable(f"player_{i}", cat="Binary") for i in players.index}

# Objective: maximize total points
model += lpSum(players.loc[i, 'total_points'] * player_vars[i] for i in players.index)

# Constraints
model += lpSum(players.loc[i, 'price'] * player_vars[i] for i in players.index) <= 100
model += lpSum(player_vars[i] for i in players.index) == 15

# Position constraints
for pos, count in {1:2, 2:5, 3:5, 4:3}.items():  # 1=GK,2=DEF,3=MID,4=FWD
    model += lpSum(player_vars[i] for i in players.index if players.loc[i,'element_type']==pos) == count

model.solve()

# Extract best squad
selected = [players.loc[i, ['web_name','price','total_points']] for i in players.index if player_vars[i].value() == 1]
best_squad = pd.DataFrame(selected)
best_squad

,web_name,price,total_points
0,Raya,5.5,10
5,Saliba,6.0,9
6,Calafiori,5.5,13
125,Semenyo,7.1,15
208,O'Riley,5.5,10
230,Cucurella,6.0,9
429,Lewis,5.0,11
444,Reijnders,5.6,10
447,Haaland,14.0,13
546,Wood,7.5,13


In [18]:
import requests, pandas as pd
import pprint

# Create a PrettyPrinter instance
pp = pprint.PrettyPrinter(indent=2)

url = "https://fantasy.premierleague.com/api/bootstrap-static/"
data = requests.get(url).json()

players = pd.DataFrame(data['elements'])
teams = pd.DataFrame(data['teams'])

In [21]:
for c in players.columns:
  print(c)

can_transact
can_select
chance_of_playing_next_round
chance_of_playing_this_round
code
cost_change_event
cost_change_event_fall
cost_change_start
cost_change_start_fall
dreamteam_count
element_type
ep_next
ep_this
event_points
first_name
form
id
in_dreamteam
news
news_added
now_cost
photo
points_per_game
removed
second_name
selected_by_percent
special
squad_number
status
team
team_code
total_points
transfers_in
transfers_in_event
transfers_out
transfers_out_event
value_form
value_season
web_name
region
team_join_date
birth_date
has_temporary_code
opta_code
minutes
goals_scored
assists
clean_sheets
goals_conceded
own_goals
penalties_saved
penalties_missed
yellow_cards
red_cards
saves
bonus
bps
influence
creativity
threat
ict_index
clearances_blocks_interceptions
recoveries
tackles
defensive_contribution
starts
expected_goals
expected_assists
expected_goal_involvements
expected_goals_conceded
influence_rank
influence_rank_type
creativity_rank
creativity_rank_type
threat_rank
threat_rank_